In [1]:
import glob
import os
import pandas as pd
import numpy as np
import pyproj
from tqdm import tqdm
from multiprocessing import Pool
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
df = pd.read_excel('../外部資料集/1.1.2低收入戶戶數及人數按鄉鎮市區別分112Q2.ods', engine='odf', skiprows=[i for i in range(6)], sheet_name=0)
#df = df.drop_duplicates(subset=['地址'], keep='first')
#df = df.reset_index()


In [3]:
use_cols = ['區域別', '區域別_英文', '總戶數', '總戶數男', '總戶數女', '總人數']
df.columns = use_cols + df.columns.tolist()[6:]
df = df[use_cols].copy()
df

,區域別,區域別_英文,總戶數,總戶數男,總戶數女,總人數
0,總計,Grand-Total,142607.0,88110.0,54497.0,276649.0
1,新北市,New Taipei City,17242.0,10538.0,6704.0,32550.0
2,板橋區,Banciao District,1208.0,753.0,455.0,2021.0
3,三重區,Sanchong District,2185.0,1368.0,817.0,3724.0
4,中和區,Jhonghe District,1628.0,966.0,662.0,2923.0
...,...,...,...,...,...,...
388,北竿鄉,Beigan Township,14.0,12.0,2.0,26.0
389,莒光鄉,Jyuguang Township,5.0,3.0,2.0,7.0
390,東引鄉,Dongyin Township,3.0,2.0,1.0,9.0
391,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
all_counties = ['新北市',
 '台北市',
 '桃園市',
 '台中市',
 '台南市',
 '高雄市',
 '宜蘭縣',
 '新竹縣',
 '苗栗縣',
 '彰化縣',
 '南投縣',
 '雲林縣',
 '嘉義縣',
 '屏東縣',
 '臺東縣',
 '花蓮縣',
 '澎湖縣',
 '基隆市',
 '新竹市',
 '嘉義市',
 '金門縣',
 '連江縣']

infos = []
city1 = ''
for i, row in df.iterrows():
    try:
        name = row['區域別'].strip()
        low_income_home_cnt = int(row['總戶數'])
        low_income_people_cnt = int(row['總人數'])
        
    except:
        print('skiping', name)
        continue
    if name.endswith('市') or name.endswith('縣'):
        name = name.strip().replace(' ', '').replace('臺北市', '台北市').replace('臺中市', '台中市').replace('臺南市', '台南市')
        if name in all_counties:
            city1 = name
            continue
    if city1 == '':
        continue
    infos.append({
        '縣市': city1,
        '鄉鎮市區': name,
        '低收入戶戶數': low_income_home_cnt,
        '低收入戶人口數': low_income_people_cnt,
    })
df_low_income = pd.DataFrame(infos)
df_low_income

skiping 東引鄉
skiping 更新日期:2023/8/31


,縣市,鄉鎮市區,低收入戶戶數,低收入戶人口數
0,新北市,板橋區,1208,2021
1,新北市,三重區,2185,3724
2,新北市,中和區,1628,2923
3,新北市,永和區,877,1503
4,新北市,新莊區,1387,2887
...,...,...,...,...
363,金門縣,烏坵鄉,1,2
364,連江縣,南竿鄉,30,48
365,連江縣,北竿鄉,14,26
366,連江縣,莒光鄉,5,7


In [5]:
use_counties = ['台北市', '高雄市', '新北市', '桃園市', '台中市', '台南市', '苗栗縣', '新竹縣', '基隆市',
       '屏東縣', '新竹市', '宜蘭縣', '花蓮縣', '嘉義市', '金門縣', '嘉義縣', '彰化縣', '雲林縣']
df_low_income = df_low_income[df_low_income['縣市'].isin(use_counties)].reset_index(drop=True)

In [6]:
set(use_counties) - set(df_low_income['縣市'].unique().tolist())

set()

In [7]:
df_low_income = df_low_income.drop_duplicates(subset=['縣市', '鄉鎮市區'], keep='first').reset_index(drop=True)

In [8]:
df_low_income.to_csv('../外部資料集/low_income_info.csv', index=False, encoding='utf-8')